# T5 Cover Letter Generator Fine-Tuning
Fine-tune t5-base on ShashiVish/cover-letter-dataset using Transformers.

In [1]:
!pip install transformers datasets accelerate -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
from datasets import Dataset

# Load dataset
dataset = load_dataset("ShashiVish/cover-letter-dataset")
# train_data = dataset["train"].shuffle(seed=42).select(range(500))
# test_data = dataset["test"].shuffle(seed=42).select(range(100))
train_data = dataset["train"]
test_data = dataset["test"]

# Combine relevant fields into one string (train and test)
def format_example(example):
    return {
        "input_text": (
            f"generate cover letter: Job Title: {example['Job Title']}. "
            f"Preferred Qualifications: {example['Preferred Qualifications']}. "
            f"Company: {example['Hiring Company']}. "
            f"Past Experience: {example['Past Working Experience']}. "
            f"Current Experience: {example['Current Working Experience']}. "
            f"Skills: {example['Skillsets']}. "
            f"Qualifications: {example['Qualifications']}."
        ),
        "target_text": example["Cover Letter"]
    }

train_formatted = [format_example(ex) for ex in train_data]
test_formatted = [format_example(ex) for ex in test_data]

train_dataset = Dataset.from_pandas(pd.DataFrame(train_formatted))
test_dataset = Dataset.from_pandas(pd.DataFrame(test_formatted))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/816 [00:00<?, ?B/s]

(…)-00000-of-00001-b1700331af6d3576.parquet:   0%|          | 0.00/350k [00:00<?, ?B/s]

(…)-00000-of-00001-460abe60f17dbc1c.parquet:   0%|          | 0.00/158k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/813 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/349 [00:00<?, ? examples/s]

In [3]:
from transformers import TrainingArguments, Trainer

model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

max_len = 512

def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], max_length=max_len, truncation=True, padding="max_length")
    targets = tokenizer(examples['target_text'], max_length=max_len, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/813 [00:00<?, ? examples/s]

Map:   0%|          | 0/349 [00:00<?, ? examples/s]

In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    do_eval=True,
    eval_steps=500,
    logging_dir='./logs',
    logging_steps=100,
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="no",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


Step,Training Loss
100,0.341700
200,0.302800
300,0.279900
400,0.286100
500,0.291000
600,0.275900
700,0.286000
800,0.270000
900,0.267600
1000,0.278700


TrainOutput(global_step=1020, training_loss=0.2866972049077352, metrics={'train_runtime': 621.5604, 'train_samples_per_second': 6.54, 'train_steps_per_second': 1.641, 'total_flos': 2475413825126400.0, 'train_loss': 0.2866972049077352, 'epoch': 5.0})

In [9]:
# Save the trained model
model.save_pretrained("t5_cover_letter_model")

# Save the tokenizer as well (required for text generation)
tokenizer.save_pretrained("t5_cover_letter_model")


('t5_cover_letter_model/tokenizer_config.json',
 't5_cover_letter_model/special_tokens_map.json',
 't5_cover_letter_model/spiece.model',
 't5_cover_letter_model/added_tokens.json')

In [11]:
!zip -r t5_cover_letter_model.zip t5_cover_letter_model
from google.colab import files
files.download("t5_cover_letter_model.zip")


updating: t5_cover_letter_model/ (stored 0%)
updating: t5_cover_letter_model/config.json (deflated 63%)
updating: t5_cover_letter_model/generation_config.json (deflated 29%)
updating: t5_cover_letter_model/tokenizer_config.json (deflated 94%)
updating: t5_cover_letter_model/spiece.model (deflated 48%)
updating: t5_cover_letter_model/model.safetensors (deflated 8%)
updating: t5_cover_letter_model/added_tokens.json (deflated 83%)
updating: t5_cover_letter_model/special_tokens_map.json (deflated 85%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_cover_letter(example):
    input_text = example["input_text"]
    input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=512).input_ids.to(device)
    outputs = model.generate(input_ids=input_ids, max_length=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Try on the first test example
print("Generated Cover Letter:\n")
print(generate_cover_letter(test_formatted[0]))


Generated Cover Letter:

Dear Hiring Manager, I am writing to express my interest in the Data Scientist position at XYZ Corporation. With my strong analytical and machine learning skills, I believe I am well-suited for this role. In my current position as a Machine Learning Engineer at DEF Company, I have gained experience in deploying and deploying machine learning models. I have also successfully used my expertise in Python, R, scikit-learn, Keras, and Tensorflow. I have also worked with Python and R to develop and ship data science based personalization services and recommendation systems. I hold a BSc in Computer Science and have 5+ years of experience in data science and machine learning engineering. My passion for data science and machine learning, combined with my passion for delivering high quality products, make me an ideal candidate for this position. I am excited about the opportunity to contribute to XYZ Corporation and contribute to its data science mission. Thank you for 

In [6]:
def generate_from_resume_and_jd(resume_text, jd_text):
    input_text = (
        f"generate cover letter: Resume: {resume_text} "
        f"Job Description: {jd_text}"
    )
    input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=512).input_ids.to(device)
    outputs = model.generate(input_ids=input_ids, max_length=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example input (replace with real data)
resume = '''Indrani Borra
 +1 945-544-2209 | indrani.borra@utdallas.edu | linkedin.com/in/indrani09 | github.com/IndraniBorra
 Technical Skills
 Languages: Java, Python, JavaScript, HTML5/CSS, C++,type script
 Frameworks: Grid, Flexbox, Media Queries, Springboot, AWS Amplify, MUnit, JUnit
 JavaScript Libraries: React, Node.js
 Developer Tools: Cloud Hub 2.0, Visual Studio Code, PyCharm, IntelliJ, Eclipse, Jira, Confluence, Anypoint platform
 Database: MySQL , Oracle, DynamoDB
 Source control: Git, BitBucket, AWS Code Commit
 Server & Build Tools: Maven, Gradle, Jenkins, Gitlab, AWS ECS, , AWS Lambda, AWS AppSync
 Education
 The University of Texas at Dallas
 Master of Science in Computer Science
 Jawaharlal Nehru Technological University
 Bachelor of Technology in Computer Science
 Experience
 Aug. 2024– May 2026
 Richardson, TX
 Aug. 2019– May 2023
 Andhra Pradesh,India
 Senior Associate Consultant
 InvenioLSI
 Jan 2023– July 2024
 Hyderabad, India
 • Developed an AI Assistant using the Spring framework for OData operations, leveraging Natural Language
 Understanding (NLU) to deliver context-aware, intent-based responses. Integrated web and PDF data scraping to
 enhance the conversational experience and improve user satisfaction.
 • Designed and deployed REST and SOAP APIs in MuleSoft, adhering to API-led architecture to meet business
 requirements. Conducted rigorous unit testing and optimized performance by monitoring logs in real-time using the
 ELK stack, reducing system downtime by 15%
 • Automated the code review process by developing a code review agent for the mule platform that adhered to
 industry-standard coding practices. This automation reduced review time by 70%, enabling faster project
 completion and improved code quality.
 • Created a reusable API development template, also published on Anypoint Exchange for external customers. The
 template reduced API development time by 25%, standardizing processes across teams and enhancing overall
 productivity.
 • Acquired in-depth knowledge of OAuth 2.0 and API security. Implemented various OAuth mechanisms, including
 custom authentication and policy applications, which strengthened data security and streamlined API
 authentication.
 Certifications
 • AWS Certified Machine Learning Engineer– Associate
 • MuleSoft Certified Developer– Integration and API Associate
 Projects
 AI-Powered Homework Helper | Python, Amazon Bedrock, NLP, Streamlit
 Dec 2024– Feb 2024
 • Developed an AI-powered homework helper using Amazon Bedrock’s Titan text-generation model to provide
 real-time, step-by-step solutions and explanations for student queries across subjects like math, science, and
 language arts. Implemented multi-modal input (text, image, voice) and adaptive learning features to personalize
 responses based on user comprehension levels, enhancing educational accessibility and efficiency. Built a
 user-friendly interface with Streamlit and integrated quality-assurance agents for solution accuracy.
Loan Approval Prediction | Python, Machine Learning Models, scikit-learn
 Oct 2024– Dec 2024
 • Developed a high-accuracy loan approval prediction model using Python and Scikit-learn to predict loan approval
 outcomes from a dataset of 45,000 applications. Implemented and evaluated multiple classification algorithms,
 including Logistic Regression, Random Forest, and XGBoost, achieving an accuracy rate of 99.67% with
 XGBClassifier. Optimized model performance through feature engineering and hyperparameter tuning, enhancing
 decision-making accuracy for financial institutions.
 Global Eats DataBase Development | SQL, Python, HTML5, CSS, Flexbox, Grid
 Sept 2024- Nov 2024
 • Developed a relational database for a global restaurant platform using SQL and Python, managing key modules
 such as Users, Employees, Restaurants, Menu Items, Orders, Payments, and Reviews. Normalized tables to 3NF,
 created SQL views for insights like top customers and popular restaurants, and wrote queries for detailed data
 analysis. Integrated a dashboard using HTML5, CSS, and React to visualize these insights, enhancing operational
 efficiency and ensuring reliable data storage and retrieval.
 Mule Custom OAuth| MuleSoft, OAuth 2.0, Git, AnyPoint Platform
 Feb 2024– March 2024
 • Designed and implemented a custom Mule OAuth authentication using client credentials grant type, securing APIs
 across the experience layer. Applied dynamic security checks and utilized API auto-discovery for policy
 enforcement, improving data confidentiality by 30% and reducing security risks.
 TideandTravels Agency Website | AWS Amplify, DynamoDB, React with type script March 2023– June 2023
 • Developed the TidesandTravels Agency Website using AWS Amplify, DynamoDB, AppSync, Lambda, Cognito,
 ECS Containers, and React with TypeScript, integrating Amadeus MPTB SOAP APIs for flight booking, flight
 search, pricing, PNR creation, and ticketing. while leveraging AWS services for real-time data synchronization,
 secure authentication, scalable backend logic, and fast, responsive front-end deployment.'''
job_description = "We are looking for a Python developer with strong skills in backend development, Flask or Django, and experience working with cloud platforms like AWS."

# 🚀 Generate and print
cover_letter = generate_from_resume_and_jd(resume, job_description)
print("Generated Cover Letter:\n")
print(cover_letter)


Generated Cover Letter:

Dear Hiring Manager, I am writing to express my interest in the Indrani Borra position at the University of Texas at Dallas. I am proficient in JavaScript and C++, with a focus on the AI Assistant model. I have a deep understanding of the AI ecosystem and have a deep understanding of the AI ecosystem. I am passionate about leveraging the Springboot framework to deliver context-aware, intent-based responses. I have also developed and deployed an AI Assistant using the Spring framework, adhering to AWS ECS, and AWS Lambda. I am a professional and have a strong working relationship with the AWS Code Framework. I am also a strong understanding of the AWS APIs and have a deep understanding of the AWS Code Commit Server and build a strong foundation in the development of AI Architectures. I am excited about the opportunity to contribute to the success of the AWS Code Commit Server and I am excited about the opportunity to contribute to the AWS Code Commit Server and 

In [ ]:
!pip install spacy -q
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
  !pip install PyMuPDF -q  # for fast and accurate PDF text extraction


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 101.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import fitz  # PyMuPDF

def extract_text_from_pdf(uploaded_file):
    doc = fitz.open(stream=uploaded_file, filetype="pdf")
    text = ""
    for page in doc:
        text += page.get_text()
    return text.strip()

# Upload PDFs
print("Please upload your RESUME PDF:")
resume_file = files.upload()
resume_text = extract_text_from_pdf(next(iter(resume_file.values())))

print("\nPlease upload the JOB DESCRIPTION PDF:")
jd_file = files.upload()
jd_text = extract_text_from_pdf(next(iter(jd_file.values())))

print("\nText extracted from both documents.")


📄 Please upload your RESUME PDF:


Saving FW-SWE-I-JD-2.pdf to FW-SWE-I-JD-2 (1).pdf

📄 Please upload the JOB DESCRIPTION PDF:


Saving FW-SWE-I-JD-2.pdf to FW-SWE-I-JD-2 (2).pdf

✅ Text extracted from both documents.


In [ ]:
import re

def clean_text(text):
    text = re.sub(r"\n{2,}", "\n", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

resume_text_clean = clean_text(resume_text)
jd_text_clean = clean_text(jd_text)


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load embedding model (small & fast)
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to compute match
def compute_resume_jd_similarity(resume_text, jd_text):
    embeddings = sbert_model.encode([resume_text, jd_text], convert_to_tensor=True)
    similarity = util.cos_sim(embeddings[0], embeddings[1]).item()
    return round(similarity * 100, 2)


In [ ]:
match_score = compute_resume_jd_similarity(resume_text_clean, jd_text_clean)
print(f"📊 Match Score: {match_score}%")

if match_score > 80:
    print("Strong match! Your resume aligns well with the job requirements.")
elif match_score > 60:
    print("Moderate match. Consider emphasizing required skills.")
else:
    print("Weak match. Consider tailoring your resume to better fit the job.")


📊 Match Score: 100.0%
✅ Strong match! Your resume aligns well with the job requirements.
